In [1]:
from __future__ import print_function
import argparse

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [3]:
import numpy as np

In [4]:
PREFER_CUDA = True

In [5]:
use_cuda = PREFER_CUDA and torch.cuda.is_available()

In [6]:
if not(use_cuda == PREFER_CUDA):
    print('CUDA SETUP NOT AS EXCEPTED')
else:
    print('OK')

OK


In [7]:
# args.seed
batch_size = 100
nb_epochs = 5

vocab_size = 250
n_words = 3
n_embed = 50
n_hidden = 200
n_output = vocab_size

learning_rate = 0.1
momentum = 0.9

In [8]:
embedding = nn.Embedding(vocab_size, 8)
# a batch of 2 samples of 4 indices each
input = Variable(torch.LongTensor([[11,20,4],[30,10,20],]))
print(embedding(input))

Variable containing:
(0 ,.,.) = 
  0.7362  2.0128  1.5946 -1.1494  0.5511 -1.8720  0.6247 -1.3850
 -1.0243 -1.2624 -1.0411 -2.0303 -0.5651 -0.2828 -1.5301  1.5863
  0.3019  0.5329  1.3229 -0.7000  0.2968  0.3983  1.2594 -1.1148

(1 ,.,.) = 
 -2.1298  1.5368 -0.4870 -1.4527 -0.5174 -1.2402  0.6551  1.4828
  1.6022  0.6770  1.0320  0.0512 -0.9889 -0.0280  1.2209 -0.3387
 -1.0243 -1.2624 -1.0411 -2.0303 -0.5651 -0.2828 -1.5301  1.5863
[torch.FloatTensor of size 2x3x8]



In [9]:
'''
class NLLLoss(_WeightedLoss):
    """The negative log likelihood loss. It is useful to train a classification problem with n classes

    If provided, the optional argument `weights` should be a 1D Tensor assigning
    weight to each of the classes.

    This is particularly useful when you have an unbalanced training set.

    The input given through a forward call is expected to contain log-probabilities
    of each class: input has to be a 2D Tensor of size `(minibatch, n)`

    Obtaining log-probabilities in a neural network is easily achieved by
    adding a  `LogSoftmax`  layer in the last layer of your network.

    You may use `CrossEntropyLoss`  instead, if you prefer not to add an extra layer.

    The target that this loss expects is a class index `(0 to N-1, where N = number of classes)`

    The loss can be described as::

        loss(x, class) = -x[class]
'''
loss_function = nn.NLLLoss()
#loss_function = nn.CrossEntropyLoss()

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convert word into vectors
        # see documentation : http://pytorch.org/docs/nn.html#torch.nn.Embedding
        self.W2E = nn.Embedding(vocab_size, n_embed, padding_idx=None, max_norm=None, norm_type=2)
        self.E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
        self.H2O = nn.Linear(n_hidden, n_output, bias=True)
        #self.conv2_drop = nn.Dropout2d()

    def forward(self, words):
        wvectors = self.W2E(words)
        # WARNING : wvectors may need a reshape operation ???
        wvectorsR = wvectors.resize(words.size(0), n_words*n_embed)
        h_activation = self.E2H(wvectorsR)
        h = F.sigmoid(h_activation)
        out_activation = self.H2O(h)
        #x = F.relu(self.fc1(x))
        #x = F.dropout(x, training=self.training)
        #return F.softmax(out_activation) # WARNING : or log_softmax ???
        return F.log_softmax(out_activation) # WARNING : or log_softmax ???

model = Net()
if use_cuda:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [11]:
from utilities import load_data
[train_input, train_target, valid_input, valid_target, test_input, test_target, vocab] = load_data(100)
train_input.shape

(3, 100, 3725)

In [12]:
TEST = True
if TEST:
    words = Variable(torch.LongTensor([[1,4,2], [13,44,3]]))
    W2E = nn.Embedding(vocab_size, n_embed, padding_idx=None, max_norm=None, norm_type=2)
    E2H = nn.Linear(n_words*n_embed, n_hidden, bias=True)
    H2O = nn.Linear(n_hidden, n_output, bias=True)
    #
    wvectors = W2E(words)
    #print(wvectors.size())
    wvectorsR = wvectors.resize(words.size(0), n_words*n_embed)
    #print(wvectorsR.size())
    h_activation = E2H(wvectorsR)
    h = F.sigmoid(h_activation)
    out_activation = H2O(h)
    print(out_activation.size())

torch.Size([2, 250])


In [13]:
if TEST:
    batch_idx = 0
    data = torch.LongTensor(train_input[:,:,batch_idx].T.astype('int'))
    print(data.size())
    target = torch.LongTensor(train_target[:,:,batch_idx].T.astype('int'))
    print(target.size())
    if use_cuda:
        data = data.cuda()
        target = target.cuda()
    data, target = Variable(data), Variable(target)
    optimizer.zero_grad()
    output = model(data)

torch.Size([100, 3])
torch.Size([100, 1])


In [14]:
print(train_target.T.astype('int').min())
print(train_target.T.astype('int').max())

0
249


In [20]:
nb_batches = 50

def train(epoch):
    model.train()
    for batch_idx in range(nb_batches):
        data = torch.LongTensor(train_input[:,:,batch_idx].T.astype('int'))
        target = torch.LongTensor(np.squeeze(train_target[:,:,batch_idx].T.astype('int'))) # WARNING : squeeze
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        #print(data.size())
        #print(target.size())
        optimizer.zero_grad()
        output = model(data)
        #print(output.size())
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        if ((batch_idx % 10 == 0) or (batch_idx == nb_batches-1)):
            print('Train Epoch: {} [{}]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), loss.data[0]))

In [16]:
# WARNING : code to complete based on train function
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
for e in range(30):
    train(e)

Train Epoch: 0 [0]	Loss: 1.817124
Train Epoch: 0 [1000]	Loss: 1.772211
Train Epoch: 0 [2000]	Loss: 1.641063
Train Epoch: 0 [3000]	Loss: 1.661208
Train Epoch: 0 [4000]	Loss: 1.976789
Train Epoch: 0 [4900]	Loss: 1.597174
Train Epoch: 1 [0]	Loss: 1.718200
Train Epoch: 1 [1000]	Loss: 1.683529
Train Epoch: 1 [2000]	Loss: 1.562136
Train Epoch: 1 [3000]	Loss: 1.591454
Train Epoch: 1 [4000]	Loss: 1.869167
Train Epoch: 1 [4900]	Loss: 1.514817
Train Epoch: 2 [0]	Loss: 1.628178
Train Epoch: 2 [1000]	Loss: 1.596057
Train Epoch: 2 [2000]	Loss: 1.483405
Train Epoch: 2 [3000]	Loss: 1.512661
Train Epoch: 2 [4000]	Loss: 1.765398
Train Epoch: 2 [4900]	Loss: 1.437800
Train Epoch: 3 [0]	Loss: 1.546964
Train Epoch: 3 [1000]	Loss: 1.510065
Train Epoch: 3 [2000]	Loss: 1.406760
Train Epoch: 3 [3000]	Loss: 1.434295
Train Epoch: 3 [4000]	Loss: 1.668755
Train Epoch: 3 [4900]	Loss: 1.365084
Train Epoch: 4 [0]	Loss: 1.471082
Train Epoch: 4 [1000]	Loss: 1.429088
Train Epoch: 4 [2000]	Loss: 1.335451
Train Epoch: 4 [